<a href="https://colab.research.google.com/github/Adithyan773/Google-colab-models/blob/main/twitter_sentimental_analysis_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install opendatasets

In [5]:
import opendatasets as od
import pandas as pd

In [6]:
od.download('https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: asd
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis


100%|██████████| 1.99M/1.99M [00:01<00:00, 1.84MB/s]


In [7]:
df = pd.read_csv('/content/twitter-entity-sentiment-analysis/twitter_training.csv')

In [8]:
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [9]:
df.columns=['id','information','type','text']

In [10]:
df.head()

,id,information,type,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['type'] = encoder.fit_transform(df['type'])

In [12]:
df['type'].unique()

array([3, 2, 1, 0])

In [ ]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

In [ ]:
!pip install emoji

In [ ]:
import string,re,nltk
import emoji
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')

In [ ]:

porter_stemmer = PorterStemmer()

In [ ]:
string.punctuation
all_stopwords = stopwords.words('english')
def remove_punctuations(v2):
    v2 = str(v2)
    for punctuation in string.punctuation:
        v2 = v2.replace(punctuation, '')
    return v2

def tokenization(text):
    return re.split(r'\s+', str(text).strip())

def remove_stopwords(text):
    stopwords_set = set(all_stopwords)
    return [i for i in text if i not in stopwords_set and i != '']

def lemmatizer(text):
    lemmatizer = WordNetLemmatizer()
    if not isinstance(text, list) or len(text) == 0:
        return []
    return [lemmatizer.lemmatize(word) for word in text]

def remove_numbers(text_list):
    text = ' '.join(text_list)
    text = re.sub(r'\d+', '', text)
    return text.split()

def remove_links(text):
    text = re.sub(r'http[s]?://\S+', '', str(text))
    return text

def deimogize(text):
    return emoji.demojize(str(text))

In [ ]:
df['text'] = df['text'].fillna('')
df['text'] = df['text'].apply(remove_links)
df['text'] = df['text'].apply(deimogize)
df['text'] = df['text'].apply(remove_punctuations)
df['text'] = df['text'].apply(lambda x: x.lower())
df['text'] = df['text'].apply(tokenization)
df['text'] = df['text'].apply(remove_stopwords)
df['text'] = df['text'].apply(remove_numbers)
df2 = df['text'].apply(lemmatizer)
df2 = df2.apply(' '.join)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(df2).toarray()

In [ ]:
tfidfconverter.get_feature_names_out()[:150]

In [ ]:
qy = df['type']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

sv = svm.SVC(kernel='rbf')

In [ ]:
sv.fit(X_train, y_train)

In [ ]:
y_pred = sv.predict(X_test)

print("Accuracy:",accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))